# **Kaggle, competizione Titanic**

Setup del notebook, con import delle librerie principali e installazione del pacchetto '*opendatasets*', in modo da fare l'import del dataset direttamente da kaggle, senza doverlo scaricare

In [ ]:
!pip install opendatasets

import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix

import numpy as np
import pandas as pd
import opendatasets as od #Per import dataset direttamente da kaggle

Effettivo import del dataset. **ATTENZIONE:** richiede username e password dell'account kaggle, credenziali scaricabili dalla scheda personale dell'account kaggle.

In [ ]:
od.download("https://www.kaggle.com/competitions/titanic/data?select=train.csv")

Caricamente dei dataset(con una scarsa scelta dei nomi, a quanto pare)

In [ ]:
# carica il dataset di train in un file
dataset0 =('./titanic/train.csv')
dataset1 = ('./titanic/test.csv')

Data pre-processing. Eliminazione delle colonne(per me) inutili, sostituzione dei valori qualificativi con valori quantitativi ed estrazione della ground_truth per la variabile target.

In [ ]:
orig_train_dataset = pd.read_csv(dataset0)
train_dataset = orig_train_dataset.copy()

#Cancello le colonne inutili
train_dataset = train_dataset.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis = 1)

#Sostituisco i valori null con la media dei valori stessi
train_dataset['Age'] = train_dataset['Age'].fillna(train_dataset['Age'].mean())
train_dataset['Embarked'] = train_dataset['Embarked'].fillna(train_dataset['Embarked'].mode()[0])

#Sostituisco i valori descrittivi con dei corrispettivi quantitativi
train_dataset.Sex.replace('male', 0, inplace=True)
train_dataset.Sex.replace('female', 1, inplace=True)

train_dataset.Embarked.replace('S', 0, inplace=True)
train_dataset.Embarked.replace('C', 1, inplace=True)
train_dataset.Embarked.replace('Q', 2, inplace=True)

print(train_dataset)

# Ottengo la "ground truth" per i singoli campioni
train_y = train_dataset.pop("Survived")
print(train_y)

     Survived  Pclass  Sex        Age  SibSp  Parch     Fare  Embarked
0           0       3    0  22.000000      1      0   7.2500         0
1           1       1    1  38.000000      1      0  71.2833         1
2           1       3    1  26.000000      0      0   7.9250         0
3           1       1    1  35.000000      1      0  53.1000         0
4           0       3    0  35.000000      0      0   8.0500         0
..        ...     ...  ...        ...    ...    ...      ...       ...
886         0       2    0  27.000000      0      0  13.0000         0
887         1       1    1  19.000000      0      0  30.0000         0
888         0       3    1  29.699118      1      2  23.4500         0
889         1       1    0  26.000000      0      0  30.0000         1
890         0       3    0  32.000000      0      0   7.7500         2

[891 rows x 8 columns]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 8

Idem per quanto riguarda il dataset di test.

In [ ]:
orig_test_dataset = pd.read_csv(dataset1)
test_dataset = orig_test_dataset.copy()

#Cancello le colonne inutili
test_dataset = test_dataset.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis = 1)

#Sostituisco i valori null con la media dei valori stessi
test_dataset['Age'] = test_dataset['Age'].fillna(test_dataset['Age'].mean())
test_dataset['Embarked'] = test_dataset['Embarked'].fillna(test_dataset['Embarked'].mode()[0])

#Sostituisco i valori descrittivi con dei corrispettivi quantitativi
test_dataset.Sex.replace('male', 0, inplace=True)
test_dataset.Sex.replace('female', 1, inplace=True)

test_dataset.Embarked.replace('S', 0, inplace=True)
test_dataset.Embarked.replace('C', 1, inplace=True)
test_dataset.Embarked.replace('Q', 2, inplace=True)

print(test_dataset)

     Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
0         3    0  34.50000      0      0    7.8292         2
1         3    1  47.00000      1      0    7.0000         0
2         2    0  62.00000      0      0    9.6875         2
3         3    0  27.00000      0      0    8.6625         0
4         3    1  22.00000      1      1   12.2875         0
..      ...  ...       ...    ...    ...       ...       ...
413       3    0  30.27259      0      0    8.0500         0
414       1    1  39.00000      0      0  108.9000         1
415       3    0  38.50000      0      0    7.2500         0
416       3    0  30.27259      0      0    8.0500         0
417       3    0  30.27259      1      1   22.3583         1

[418 rows x 7 columns]


Ultimo controllo per valori '*NaN*', just in case.

In [ ]:
test_dataset[np.isnan(test_dataset) == True] = 0

Dichiarazione del modello(in questo caso un semplice modello di regressione logistica, in quanto il dominio della variabile target è [0,1]) e del componente di standardizzazione, in modo da normalizzare i dati(altro piccolo step di data-preprocessing).

In [ ]:
classifier = LogisticRegression()
scaler = StandardScaler()
model = Pipeline([('standardize', scaler),
                    ('classifier', classifier)])

Addestramento del modello con i dati di train.

In [ ]:
model.fit(train_dataset, train_y)

Pipeline(steps=[('standardize', StandardScaler()),
                ('classifier', LogisticRegression())])

Predizione della variabile target sui dati di train per avere un insight sulla correttezza e l'
efficienza del modello.

In [ ]:
train_result = model.predict(train_dataset)
train_result_prob = model.predict_proba(train_dataset)

train_accur = accuracy_score(train_y, train_result) * 100

80% di accuratezza, non male. Si attende con ansia la distruzione di questo risultato da parte del dataset di test.

In [ ]:
print(train_accur)

80.13468013468014


Dato che abbiamo già preprocessato i dati di test, possiamo direttamente darli in pasto al modello per ottenerne le predizioni.

In [ ]:
test_result = model.predict(test_dataset)
test_result_prob = model.predict_proba(test_dataset)

In [ ]:
#print(test_result)
print(test_result_prob)

Dato che è una competizione Kaggle, salvataggio delle predizioni per la submission

In [ ]:
submission["PassengerId"] = dataset_tmp["PassengerId"]
submission.set_index("PassengerId")
submission.to_csv("/content/drive/MyDrive/submission.csv", index = False, encoding='utf-8-sig')